# Tweets Scraping

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import re
import requests
import json

## Extract and parse original data

In [2]:
df = pd.read_excel('ProjectList.xls')

# get rid of spaces inside ICO names
name_strip = []
for name in df['name'].tolist():
    name_strip.append(name.replace(' ', ''))
df.name = name_strip

#df.columns
df = df[['name', 'PreICOtimestart', 'PreICOtimeend', 'preICOstart', 'preICOend', 'ICOTimestart', 'ICOTimeend', 'ICOstart', 'ICOend']]
df = df.astype(object).where(df.notnull(), None) # change data type to None
df.shape
#df.head(10)

# parse date into the format when scraping on twitter
## ordinal date format
def ordinal_parser(col):
    date = []
    for item in col:
        if item != None:
            date.append(datetime.strptime(item.replace(item.split(' ')[0], re.findall('.\d*', item)[0]), '%d %b %Y').strftime('%Y-%m-%d'))
        else:
            date.append(None)
    return date
### these four attributes are in ordinal format due to the source website
df.preICOstart = ordinal_parser(df.preICOstart)
df.preICOend = ordinal_parser(df.preICOend)
df.ICOstart = ordinal_parser(df.ICOstart)
df.ICOend = ordinal_parser(df.ICOend)

## date time format
def datetime_parser(col):
    date = []
    for item in col:
        if item != None:
            date.append(datetime.date(item).strftime('%Y-%m-%d'))
        else:
            date.append(None)
    return date
### these four attributes are in datetime format due to another source website
df.PreICOtimestart = datetime_parser(df.PreICOtimestart)
df.PreICOtimeend = datetime_parser(df.PreICOtimeend)
df.ICOTimestart = datetime_parser(df.ICOTimestart)
df.ICOTimeend = datetime_parser(df.ICOTimeend)

#df.head(10)

(1876, 9)

## Tweets in pre-sale stage
### 1. data preparation

In [3]:
# navigate to ICOs which have date info in their pre-sale stage
df_pre = df.iloc[:,0:5]
#df_pre.head(10)

# check if there is any observation that contains infomation from both sources
## start
for ind1 in df_pre[df_pre.PreICOtimestart.isna() == False].index:
    for ind2 in df_pre[df_pre.preICOstart.isna() == False].index:
        if ind1 == ind2:
            print('match found in starts')
            break
## end
for ind1 in df_pre[df_pre.PreICOtimeend.isna() == False].index:
    for ind2 in df_pre[df_pre.preICOend.isna() == False].index:
        if ind1 == ind2:
            print('match found in ends')
            break

# no overlaps, then merge starts and ends together
d1 = df_pre[(df_pre.PreICOtimestart.isna() == False) & (df_pre.PreICOtimeend.isna() == False)].iloc[:, 0:3]
d2 = df_pre[(df_pre.preICOstart.isna() == False) & (df_pre.preICOend.isna() == False)].iloc[:, [0, 3, 4]]
#d1.head()
#d2.head()
name = pd.concat([d1.name, d2.name], ignore_index = True)
start = pd.concat([d1.PreICOtimestart, d2.preICOstart], ignore_index = True)
end = pd.concat([d1.PreICOtimeend, d2.preICOend], ignore_index = True)

df_pre = pd.concat([name, start, end], axis = 1)
df_pre.columns = ['name', 'pre_start', 'pre_end']
df_pre = df_pre.sort_values(by = ['pre_start', 'pre_end'], ascending = [True, True]).reset_index(drop = True)

df_pre.shape
df_pre.head()

(807, 3)

,name,pre_start,pre_end
0,BitparkCoin,2016-07-25,2016-08-15
1,XinFin,2017-06-01,2017-07-31
2,Cofound.it,2017-06-05,2017-06-07
3,Populous,2017-06-08,2017-07-08
4,Kapu,2017-06-15,2017-08-31


### 2. scrape tweets in pre-sale stage

In [7]:
# define a function to parse tweets html using BeautifulSoup

def getTweets(tweets_html):
    
    result=[]
    
    soup=BeautifulSoup(tweets_html, "html.parser")

    tweets=soup.select('div.js-stream-tweet')

    for t in tweets:
        ICO_name, username, text, timestamp, reply, retweet, like = '', '', '', '', '', '', ''
        
        ICO_name = row[0]
        
        select_user = t.select("span.username.u-dir b")
        if select_user != []:
            username = select_user[0].get_text()

        select_text = t.select("p.js-tweet-text")
        if select_text != []:
            text = select_text[0].get_text()
    
        time_select = t.select("small.time span.js-short-timestamp")
        if time_select != []:
            timestamp = int(time_select[0]["data-time"])
            timestamp = datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")
        
        select_stream_item_footer = t.select('span.ProfileTweet-actionCountForPresentation')
        if select_stream_item_footer != []:
            reply = select_stream_item_footer[0].get_text()
            retweet = select_stream_item_footer[1].get_text()
            like = select_stream_item_footer[3].get_text()
       
        result.append({'ICO_name':ICO_name, "user":username, "text":text, "date":timestamp, 'reply':reply, 'retweet':retweet, 'like':like})
        
    return result

In [8]:
# User agent must be defined in http request header
# a user agent is software that is acting on behalf of a user. Usually it tells the browser used.
# some websites reject requests without a user agent
headers = { 'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.'
                              '86 Safari/537.36'}

# specify parameters as a dictionary
# the search returns tweets in the decending order of time, set the max_position to the min_position of last search
min_position = '1'
min_position_updated = ''
tweets_scraped = []

for ind, row in df_pre.iterrows():
    while min_position != min_position_updated:
        min_position = min_position_updated

        payload={"f":"tweets",  # retrieve tweets
                 "q":"#ICO#" + row[0] + " since:" + row[1] + " until:" + row[2], # query string
                 "max_position": min_position} 

        # send a request with parameters and headers
        r = requests.get("https://twitter.com/i/search/timeline",\
                      params=payload, headers=headers)

        # this is equivalent to type the URL
        if r.status_code==200:
            result=r.json()

            # get html source code of tweets
            tweets_html = result['items_html']

            # Search returns tweets in the decending order of time
            # retrieve the position of the earliest tweets returned
            min_position_updated = result['min_position']
            # update tweets list
            tweets_scraped += getTweets(tweets_html)  
    
    # reset position parameter to keep the loop going        
    min_position = '1'
    min_position_updated = ''

# save results to dataframe and csv file
tweets = pd.DataFrame(tweets_scraped)
tweets.head(10)
tweets.to_csv('tweets_pre.csv', index = False)

,ICO_name,date,like,reply,retweet,text,user
0,XinFin,2017-07-11 12:57:34,,,,Join Xinfin’s #Bitcoin Ecosystem & Bridge the ...,bitcoinewsboard
1,XinFin,2017-06-15 08:31:21,,,,Join Xinfin’s Bitcoin Ecosystem & Bridge the G...,coinspectator
2,Cofound.it,2017-06-06 17:06:14,5,,4,@cofound_it Succes ICO presale 96% collected ...,CyrilGaab
3,Cofound.it,2017-06-06 01:51:45,3,,3,News from on #blockchain : #Cofound.it Token...,blockchainers
4,Cofound.it,2017-06-05 10:55:38,,,,Fresh #blockchain News: #Cofound.it Token Pre-...,blockchainers
5,Populous,2017-07-05 11:51:13,,,,Another #ICO didn't make it to public phase! #...,emiemiemij
6,Populous,2017-06-30 07:49:57,,,,Congratulations to all who got in on #Populous...,CryptoFloridian
7,Populous,2017-06-29 06:47:02,,,,#Populous #ICO already up and running driving ...,JohnFMorton
8,Populous,2017-06-24 14:29:59,2,,,#Populous #ICO SOLD out with 1.5 weeks to go i...,CryptoFloridian
9,Populous,2017-06-23 20:38:10,1,,,"#ICO, #Populous platform tackling the Multi-T...",seandotau


## Tweets in sale stage
### 1. data preparation

In [10]:
# navigate to ICOs which have date info in their sale stage
df_sale = df.iloc[:,[0, 5, 6, 7, 8]]
#df_sale.head(10)

# check if there is any observation that contains infomation from both sources
## start
for ind1 in df_sale[df_sale.ICOTimestart.isna() == False].index:
    for ind2 in df_sale[df_sale.ICOstart.isna() == False].index:
        if ind1 == ind2:
            print('match found in starts')
            break
## end
for ind1 in df_sale[df_sale.ICOTimeend.isna() == False].index:
    for ind2 in df_sale[df_sale.ICOend.isna() == False].index:
        if ind1 == ind2:
            print('match found in ends')
            break

# no overlaps, then merge starts and ends together
d1 = df_sale[(df_sale.ICOTimestart.isna() == False) & (df_sale.ICOTimeend.isna() == False)].iloc[:, 0:3]
d2 = df_sale[(df_sale.ICOstart.isna() == False) & (df_sale.ICOend.isna() == False)].iloc[:, [0, 3, 4]]
#d1.head()
#d2.head()
name = pd.concat([d1.name, d2.name], ignore_index = True)
start = pd.concat([d1.ICOTimestart, d2.ICOstart], ignore_index = True)
end = pd.concat([d1.ICOTimeend, d2.ICOend], ignore_index = True)

df_sale = pd.concat([name, start, end], axis = 1)
df_sale.columns = ['name', 'sale_start', 'sale_end']
df_sale = df_sale.sort_values(by = ['sale_start', 'sale_end'], ascending = [True, True]).reset_index(drop = True)

df_sale.shape
df_sale.head()

# check if all ICOs that have sale date also have pre-sale date
df_pre.merge(df_sale, on = 'name', how = 'inner').shape

(1755, 3)

,name,sale_start,sale_end
0,BitCrystals,2015-08-04,2015-09-03
1,Augur,2015-08-17,2015-10-01
2,DigixDAO,2016-03-29,2016-03-30
3,Waves,2016-04-12,2016-05-31
4,Newbium,2016-05-01,2016-05-31


(809, 5)

### 2. scrape tweets in sale stage

In [11]:
# similar to the metrics for scraping tweets in pre-sale stage
# User agent must be defined in http request header
# a user agent is software that is acting on behalf of a user. Usually it tells the browser used.
# some websites reject requests without a user agent
headers = { 'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.'
                              '86 Safari/537.36'}

# specify parameters as a dictionary
# the search returns tweets in the decending order of time, set the max_position to the min_position of last search
min_position = '1'
min_position_updated = ''
tweets_scraped = []

for ind, row in df_sale.iterrows():
    while min_position != min_position_updated:
        min_position = min_position_updated

        payload={"f":"tweets",  # retrieve tweets
                 "q":"#ICO#" + row[0] + " since:" + row[1] + " until:" + row[2], # query string
                 "max_position": min_position} 

        # send a request with parameters and headers
        r = requests.get("https://twitter.com/i/search/timeline",\
                      params=payload, headers=headers)

        # this is equivalent to type the URL
        if r.status_code==200:
            result=r.json()

            # get html source code of tweets
            tweets_html = result['items_html']

            # Search returns tweets in the decending order of time
            # retrieve the position of the earliest tweets returned
            min_position_updated = result['min_position']
            # update tweets list
            tweets_scraped += getTweets(tweets_html)  
    
    # reset position parameter to keep the loop going        
    min_position = '1'
    min_position_updated = ''

# save results to dataframe and csv file
tweets = pd.DataFrame(tweets_scraped)
tweets.head(10)
tweets.to_csv('tweets_sale.csv', index = False)

,ICO_name,date,like,reply,retweet,text,user
0,Waves,2016-05-30 10:30:24,2,,1,Currently Active #ICO #Crowdsales #waves #rise...,TheEtherFire
1,Waves,2016-05-29 13:23:04,7,,7,$waves https://bitcointalk.org/index.php?topic...,Vindyne8
2,Waves,2016-05-29 09:19:03,,,,Waves ico achieved 15K BTC !!!! So + 6 000 000...,samyrhf
3,Waves,2016-05-27 19:22:37,14,1,21,#Waves: 14K #BTC raised so far - Currently No4...,xbtmoney
4,Waves,2016-05-19 15:07:46,,,,#waves #ICO https://twitter.com/wavesplatform/...,JellaBoem
5,Waves,2016-05-18 13:25:13,1,,,Mycelium Bitcoin Wallet Crowdsale Closes in on...,samyrhf
6,Waves,2016-05-14 08:17:17,5,1,4,"Wanna invest some #BTC? ""Currently Active #ICO...",xbtmoney
7,Waves,2016-05-12 12:06:07,16,,24,"#Waves ICO: 8434 #BTC raised so far, already N...",xbtmoney
8,Waves,2016-05-11 21:44:56,,,,8 hours left to buy #Rise tokens with 25% bonu...,IgorKripter
9,Waves,2016-05-09 18:25:44,,,,Interesting. Are the #cryptocurrency markets i...,menace_coin
